In [1]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from mlflow.models.signature import infer_signature

In [6]:
iris = load_iris()
X = iris.data
y = iris.target
print(iris.target_names)

['setosa' 'versicolor' 'virginica']


In [7]:
print(iris.feature_names)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [4]:
print(type(X))
print(X.shape)

<class 'numpy.ndarray'>
(150, 4)


In [ ]:

def main():
    with mlflow.start_run():
        # Set a description for the run
        mlflow.set_tag("description", "Logistic Regression model for the Iris dataset")

        # Set custom tags
        mlflow.set_tag("model_type", "Logistic Regression")
        mlflow.set_tag("dataset", "Iris")
        # Load the data
        iris = load_iris()
        X = iris.data
        y = iris.target

        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Train the model
        model = LogisticRegression(max_iter=200)
        model.fit(X_train, y_train)

        # Evaluate the model
        score = model.score(X_test, y_test)
        print(f"Model accuracy: {score}")

        # Log parameters and metrics
        mlflow.log_param('max_iter', 200)
        mlflow.log_metric('accuracy', score)

        #  Log and register the model with input example and signature
        input_example = X_test[:5]  # A few example inputs
        signature = infer_signature(X_test, model.predict(X_test))
        
        mlflow.sklearn.log_model(model, 'model', signature=signature, input_example=input_example)
        mlflow.register_model(f'runs:/{mlflow.active_run().info.run_id}/model', 'IrisClassifier')

if __name__ == "__main__":
    main()


In [4]:
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from mlflow.models.signature import infer_signature

def main():
    with mlflow.start_run():
        # Load the data
        iris = load_iris()
        X = iris.data
        y = iris.target

        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Train the model
        model = LogisticRegression(max_iter=200)
        model.fit(X_train, y_train)

        # Evaluate the model
        score = model.score(X_test, y_test)
        print(f"Model accuracy: {score}")

        # Log parameters and metrics
        mlflow.log_param('max_iter', 200)
        mlflow.log_metric('accuracy', score)

        # Log and register the model with input example and signature
        input_example = X_test[:5]  # A few example inputs
        signature = infer_signature(X_test, model.predict(X_test))
        
        mlflow.sklearn.log_model(model, 'model', signature=signature, input_example=input_example)
        
        # Register the model
        model_uri = f'runs:/{mlflow.active_run().info.run_id}/model'
        registered_model = mlflow.register_model(model_uri, 'IrisClassifier')

    # Initialize the MLflow Client
    client = MlflowClient()

    # Update the description of the model version
    client.update_model_version(
        name="IrisClassifier", 
        version=registered_model.version, 
        description="Logistic Regression model trained on the Iris dataset"
    )

    # Set tags for the model version
    client.set_model_version_tag("IrisClassifier", registered_model.version, "model_type", "Logistic Regression")
    client.set_model_version_tag("IrisClassifier", registered_model.version, "dataset", "Iris")
    client.set_model_version_tag("IrisClassifier", registered_model.version, "experiment_version", "1.0")

if __name__ == "__main__":
    main()


Model accuracy: 0.9333333333333333


Registered model 'IrisClassifier' already exists. Creating a new version of this model...
Created version '3' of model 'IrisClassifier'.
